# BeautyfulSoup 네이버 영화 리뷰 스크래핑

## 네이버 영화 리뷰 스크래핑

* 네이버 영화: https://movie.naver.com/movie/point/af/list.nhn

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_movie_reviews(mcode, page_num=10): #default 10

  movie_review_df = pd.DataFrame(columns=("Title", "Score", "Review"))
  url = "https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=" +str(mcode) + "&target=after"
  idx = 0

  for _ in range(0, page_num):
    movie_page = urllib.request.urlopen(url).read()
    movie_page_soup = BeautifulSoup(movie_page, 'html.parser')

    review_list = movie_page_soup.find_all('td', {'class':'title'})
    for review in review_list:
      title = review.find('a', {'class':'movie color_b'}).get_text()
      score = review.find('em').get_text()
      review_text = review.find('a', {'class':'report'}).get('href').split(',')[2]
      movie_review_df.loc[idx] = [title, score, review_text]
      idx += 1
      print("#", end="")
      
    try:        #다음 페이지 없을 경우 예외 처리
      url = "https://movie.naver.com" + movie_page_soup.find('a', {'class':'pg_next'}).get('href')
    except:
      break

  return movie_review_df

In [ ]:
movie_review_df = get_movie_reviews(18847, 10)
movie_review_df

####################################################################################################

,Title,Score,Review
0,타이타닉,10,''
1,타이타닉,10,'상실을 아는 사람들이 살아가는 이야기'
2,타이타닉,10,''
3,타이타닉,10,''
4,타이타닉,10,'최고의 영화 말이 필요없음'
...,...,...,...
95,타이타닉,10,''
96,타이타닉,10,'미쳤네요 제 인생최고의 영화'
97,타이타닉,10,''
98,타이타닉,10,'송규 지현 소정과 함께 봐서 행복했습니다.'


## 현재상영작 영화 리뷰 스크래핑

In [ ]:
url = "https://movie.naver.com/movie/point/af/list.nhn"
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select', {'id':'current_movie'})
movies = select.find_all('option')

movies_dict = {}
for movie in movies[1:]:
  movies_dict[movie.get('value')] = movie.get_text()

movie_review_df = pd.DataFrame(columns=("Title", "Score", "Review"))
for mcode in movies_dict:
  df = get_movie_reviews(mcode, 1)
  movie_review_df = pd.concat([movie_review_df, df]) #새로 받은 df를 concat해줌

movie_review_df

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

,Title,Score,Review
0,극장판 짱구는 못말려: 신혼여행 허리케인~ 사라진 아빠!,1,'웃겨요 그냥 어이없어서'
1,극장판 짱구는 못말려: 신혼여행 허리케인~ 사라진 아빠!,10,' 말할게 없음 항상 최고'
2,극장판 짱구는 못말려: 신혼여행 허리케인~ 사라진 아빠!,10,'엄마가 주가 된건 처음 같은데 너무 감동적이였고 어른제국의 역습 급이라고 생각해...
3,극장판 짱구는 못말려: 신혼여행 허리케인~ 사라진 아빠!,10,''
4,극장판 짱구는 못말려: 신혼여행 허리케인~ 사라진 아빠!,10,'애가 좋아해서 보여주러 갔다가 제가 감동받고 재밌게 보고 왔어요 ㅎ IPTV로 ...
...,...,...,...
5,왕중왕,9,'예수를 다룬 영화중에 가장 뛰어나다...'
6,왕중왕,2,'예수 같은 왕 둔 적 없다.'
7,왕중왕,10,'예수님의 왕 자리는 대통령 이나 수상이나 황제나 천왕 의 자리가 아니...
8,왕중왕,10,'20세기가 낳은 최고의 명작'
